# Credit Score Rating Analysis

A comprehensive analysis to evaluate creditworthiness based on various factors.

## Executive Summary
This project analyzes credit data to evaluate the creditworthiness of customers. The goal is to identify patterns and relationships between various factors such as income, marital status, loan purpose, and repayment behavior. The results aim to help decision-makers make informed decisions regarding loan approvals.

### Main Objectives:
1. Data cleaning and preparation.
2. Analysis of relationships between creditworthiness and factors such as income, marital status, and loan purpose.
3. Development of a scoring system to evaluate creditworthiness.

### Results:
- The majority of customers show a low risk of loan defaults.
- The most common loan purpose is real estate acquisition.
- There are weak but significant relationships between marital status and repayment behavior.

## Table of Contents
1. [Executive Summary](#executive-summary)
2. [Data Preparation](#data-preparation)
   - [Missing Values](#missing-values)
   - [Data Types](#data-types)
   - [Duplicates](#duplicates)
3. [Data Analysis](#data-analysis)
   - [Relationship Between Marital Status and Repayment](#relationship-between-marital-status-and-repayment)
   - [Relationship Between Income and Repayment](#relationship-between-income-and-repayment)
   - [Relationship Between Loan Purpose and Repayment](#relationship-between-loan-purpose-and-repayment)
4. [Conclusion](#conclusion)

In [30]:
#pip install kagglehub

In [21]:
import pandas as pd
import kagglehub
import nltk
from nltk.stem import SnowballStemmer
english_stemmer = SnowballStemmer('english')

import scipy
import warnings 
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
path = kagglehub.dataset_download("judechristensenesq/practicum-bank-data")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\torsten.zick\.cache\kagglehub\datasets\judechristensenesq\practicum-bank-data\versions\2


In [5]:
## open the dataset
df = pd.read_csv(path + "/credit_scoring_eng.csv")

In [6]:
display(df.info(),
        df.describe(),
        df.sample(5),
        df.isnull().sum(),
        df.isnull().sum()/len(df)*100
        )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4026,1,-393.483813,37,secondary education,1,married,0,M,employee,0,24650.541,profile education
17121,2,-515.862659,40,secondary education,1,married,0,F,employee,1,41792.287,car purchase
20260,0,-221.558831,44,secondary education,1,married,0,F,civil servant,0,30011.246,building a property
10812,0,-840.340906,57,bachelor's degree,0,married,0,F,business,0,47144.256,to own a car
10465,1,-2446.672529,32,SECONDARY EDUCATION,1,civil partnership,1,M,business,0,35500.439,supplementary education


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

children             0.000000
days_employed       10.099884
dob_years            0.000000
education            0.000000
education_id         0.000000
family_status        0.000000
family_status_id     0.000000
gender               0.000000
income_type          0.000000
debt                 0.000000
total_income        10.099884
purpose              0.000000
dtype: float64

## Data Preparation
This section describes the data cleaning and preparation process.

### Processing missing values

In [ ]:
#filling the median value of days_employed and total_income for the missing values 
#in the respective columns of the dataframe


missing_values_columns = ['days_employed', 'total_income']

for col in missing_values_columns:
    df.fillna({col: df[col].median()}, inplace=True)


#Monitor the changes
df.info()

### Data type replacement
Negative values were converted to positive, and the data types were adjusted accordingly.

In [ ]:
#first, handling negative values
df[['days_employed', 'children']] = df[['days_employed', 'children']].abs()

#then, changing datatypes from float to int
df = df.astype({'days_employed': 'int', 'total_income': 'int'})
df.info()

### Processing duplicates

In [ ]:
customer_purposes = df['purpose'].unique().tolist()

def purpose_category(customer_purposes):
    if 'wedd' in customer_purposes:
        return 'wedding'
    if 'car' in customer_purposes:
        return 'car'
    if 'educat' in customer_purposes:
        return 'education'
    if 'university' in customer_purposes:
        return 'education'
    if 'hous' in customer_purposes:
        return 'real estate'
    if 'real' in customer_purposes:
        return 'real estate'
    if 'estat' in customer_purposes:
        return 'real estate'
    if 'building' in customer_purposes:
        return 'real estate'
    if 'constr' in customer_purposes:
        return 'real estate'
    if 'prop' in customer_purposes:
        return 'real estate'
    else: 
        return 'other'

df['purpose_category'] = df['purpose'].apply(purpose_category)
print(df['purpose_category'].value_counts().head())

## Data Analysis
This section analyzes the relationships between various factors and creditworthiness.

### Categorizing Data

In [10]:
### Basic settings

credit_score = 0
positive = credit_score + 1
negative = credit_score - 1

#-----------------------------------------------------------------------------
# 1. evaluate the income rating in comperisson to the total mean
#-----------------------------------------------------------------------------



df['total_income'] = pd.to_numeric(df['total_income'], errors='coerce')
df.groupby(['income_type'])['total_income'].median()
df['total_income'] = df['total_income'].astype('float')

 
def credit_scoring(total_income):

 
    if total_income <= 10000:
        return negative -1
    if total_income <= 20000:
        return negative
    if total_income <= 30000:
        return positive
    else:
        return positive +1
    


#-----------------------------------------------------------------------------
# 2. evaluate the current debt rating 
#-----------------------------------------------------------------------------
 
df['debt'] = pd.to_numeric(df['debt'], errors='coerce')
debt_lvl   = df['debt']

   
def debt_check(debt_lvl):
    if debt_lvl == 0:
        return positive 
    else:
        return negative -1

#-----------------------------------------------------------------------------
# 3. evalute the total numbers of children 
#-----------------------------------------------------------------------------

df['cildren'] = pd.to_numeric(df['children'], errors='coerce')
children_count = df['children']


def children_check(children_count):
    if children_count == 0:
        return positive +1
    if children_count <= 2:
        return positive -1
    else:
        return negative

#-----------------------------------------------------------------------------
# 4. taking the current family status into concideration
#-----------------------------------------------------------------------------


df['family_status_id'] = pd.to_numeric(df['family_status_id'], errors='coerce')
marriage_status = df['family_status_id']


def marriage_status_check(marriage_status):
    if marriage_status == 0:
        return positive +1
    if marriage_status == 1:
        return positive
    if marriage_status == 4:
        return positive -1
    if marriage_status == 3:
        return negative
    else: negative -1


#-----------------------------------------------------------------------------
# 5. taking the age into concideration
#-----------------------------------------------------------------------------


df['dob_years'] = pd.to_numeric(df['dob_years'], errors='coerce')
age = df['dob_years']


def age_check(age):
    if age >= 65:
        return negative -1
    if age >= 55 and  age < 65:
        return positive +1
    if age >= 25 and age < 55: 
        return positive +2
    if age >= 18 and age < 25:
        return positive -1
    else:
        return negative -1


#-----------------------------------------------------------------------------
# 6. Applying the custom function to evaluate a credig rating
#-----------------------------------------------------------------------------



var1  = df['total_income'].apply(credit_scoring)
var2  = df['debt'].apply(debt_check)
var3  = df['children'].apply(children_check)
var4  = df['family_status_id'].apply(marriage_status_check)
var5  = df['dob_years'].apply(age_check)

df['credit_score'] = var1+var2+var3+var4+var5


#-----------------------------------------------------------------------------
# 7. Applying result column
#-----------------------------------------------------------------------------


rating = df['credit_score']

def results(rating):
    if rating >= 9:
        return 'credit worthy. Zero to low chance for repayment fail'
    if rating >= 6 and rating < 9:
        return 'credit worthy. Small chance for repayment fail'
    if rating >= 4 and rating < 6:
        return 'Medium chance for repayment fail. Additional security deposit recommended'
    if rating < 4:
        return 'Not recommended. High risk of repayment fail by that customer'

df['result'] = df['credit_score'].apply(results)
    

print(df['result'].value_counts())

result
credit worthy. Small chance for repayment fail                               9938
Medium chance for repayment fail. Additional security deposit recommended    4516
credit worthy. Zero to low chance for repayment fail                         4055
Not recommended. High risk of repayment fail by that customer                2056
Name: count, dtype: int64


## Relationship Between Income and Repayment
No significant correlation was found between income category and repayment behavior.

Check for Questions:
1. Is there a relation between marital status and repaying a loan on time?
2. Is there a relation between income level and repaying a loan on time?
3. How do different loan purposes affect on-time repayment of the loan?

In [11]:
from scipy.stats import spearmanr, pearsonr

def loan_and_marital_status(row):
    marital_status = row['family_status']
    no_debts = row['debt']
 
    if marital_status == 'married':
        if no_debts == 0:
            return 'married and debt free'
        else:
            return 'married with debts'
    if marital_status == 'civil partnership':
        if no_debts == 0:
            return 'civil partnership and debt free'
        else:
            return 'civil partnership with debts'
    if marital_status == 'widow / widower':
        if no_debts == 0:
            return 'widow / widower and debt free'
        else:
            return 'widow / widower with debts'
    if marital_status == 'divorced':
        if no_debts == 0:
            return 'divorced and debt free'
        else:
            return 'divorced with debts'
    if marital_status == 'unmarried':
        if no_debts == 0:
            return 'unmarried and debt free'
        else:
            return 'unmarried with debts'


df['loan_and_marital_status'] = df.apply(loan_and_marital_status,axis=1)

# Calculate the Spearman correlation between family_status and debt
family_status_mapping = {status: idx for idx, status in enumerate(df['family_status'].unique())}
df['family_status_numeric'] = df['family_status'].map(family_status_mapping)

spearman_corr, p_value = spearmanr(df['family_status_numeric'], df['debt'])
print(f"Spearman correlation: {spearman_corr}, P-value: {p_value}")


Spearman correlation: 0.023444694549968198, P-value: 0.0005818472237756956


The Spearman correlation coefficient of 0.0234 shows a very weak positive correlation between marital status and debt (debt status). However, the p-value of 0.00058 is statistically significant (typically at a significance level of 0.05), which means that the relationship between these variables is not random.

Summary:

Correlation: Very weak positive.
Significance: Statistically significant.
This suggests that marital status may have a minimal but significant impact on debt status.

In [13]:
### check for relationship between low income and paying on time

df['total_income'] = pd.to_numeric(df['total_income'], errors='coerce')

df.groupby(['income_type'])['total_income'].median()
df['total_income'] = df['total_income'].astype('float')

 
def income_cat(total_income):

 
    if total_income <= 10000:
        return 'low_income'
    if total_income <= 20000:
        return 'lower_mid_income'
    if total_income <= 30000:
        return 'upper_mid_income'
    else:
        return 'very_high_income'
    
df['income_cat'] = df['total_income'].apply(income_cat)


def loan_on_income_level(row):
    income_category = row['income_cat']
    no_debts = row['debt']
 
    if income_category == 'low_income':
        if no_debts == 0:
            return 'low income and debt free'
        else:
            return 'low income with debts'
    if income_category == 'lower_mid_income':
        if no_debts == 0:
            return 'lower mid income and debt free'
        else:
            return 'lower mid income with debts'
    if income_category == 'upper_mid_income':
        if no_debts == 0:
            return 'upper mid income and debt free'
        else:
            return 'upper mid income with debts'
    if income_category == 'very_high_income':
        if no_debts == 0:
            return 'very high income and debt free'
        else:
            return 'very high income with debts'

df['debt_income_category'] = df.apply(loan_on_income_level,axis=1)

# Calculate the Spearman correlation between income category and debt
income_category_mapping = {category: idx for idx, category in enumerate(df['income_cat'].unique())}
df['income_cat_numeric'] = df['income_cat'].map(income_category_mapping)

print(df['debt_income_category'].value_counts())

spearman_corr, p_value = spearmanr(df['income_cat_numeric'], df['debt'])
print(f"Spearman correlation: {spearman_corr}, P-value: {p_value}")



debt_income_category
upper mid income and debt free    7539
lower mid income and debt free    5893
very high income and debt free    5483
low income and debt free           869
upper mid income with debts        697
lower mid income with debts        550
very high income with debts        436
low income with debts               58
Name: count, dtype: int64
Spearman correlation: 0.008348487019383469, P-value: 0.2206542790346616


The Spearman correlation coefficient of 0.0083 shows an extremely weak positive correlation between income category and debt (debt status). However, the p-value of 0.2207 is not statistically significant (typically at a significance level of 0.05), which means that the relationship between these variables could be random.

Summary:

Correlation: Extremely weak positive.
Significance: Not statistically significant.
This suggests that there is no significant relationship between income category and debt status.

## Relationship Between Loan Purpose and Repayment
The majority of customers who apply for real estate loans repay on time.

In [15]:
df['loan_purpose'] = df['purpose_category']


df.loc[df["loan_purpose"] =="real estate", "loan_purpose"] = 1 
df.loc[df["loan_purpose"] =="car", "loan_purpose"] = 2
df.loc[df["loan_purpose"] =="wedding", "loan_purpose"] = 3
df.loc[df["loan_purpose"] =="education", "loan_purpose"] = 4



def loan_vs_loan_purpose(row):
    loan_purpose = row['loan_purpose']
    no_debts = row['debt']
 
    if loan_purpose == 1:
        if no_debts == 0:
            return 'debt free and asked for real estate loan'
        else:
            return 'negative debt history and asked for real estate loan'
    if loan_purpose == 2:
        if no_debts == 0:
            return 'debt free and asked for a car loan'
        else:
            return 'negative debt history and asked for a car loan'
    if loan_purpose == 4:
        if no_debts == 0:
            return 'debt free and asked for a loan for education purposes'
        else:
            return 'negative debt history and asked for a loan for education purposes'
    if loan_purpose == 3:
        if no_debts == 0:
            return 'debt free and asked for a loan for wedding purposes'
        else:
            return 'negative debt history and asked for a loan for wedding purposes'



df['loan_vs_loan_purpose'] = df.apply(loan_vs_loan_purpose,axis=1)
print(df['loan_vs_loan_purpose'].value_counts())

loan_vs_loan_purpose
debt free and asked for real estate loan                             10058
debt free and asked for a car loan                                    3912
debt free and asked for a loan for education purposes                 3652
debt free and asked for a loan for wedding purposes                   2162
negative debt history and asked for real estate loan                   782
negative debt history and asked for a car loan                         403
negative debt history and asked for a loan for education purposes      370
negative debt history and asked for a loan for wedding purposes        186
Name: count, dtype: int64


### Conclusion
The analysis shows that most customers are creditworthy. Real estate is the most common loan purpose, and repayment behavior is largely positive.

## Final Conclusion
When analyzing a dataset, the dataset must first be cleaned and adjusted. One of the tasks of a data analyst is to prepare the data provided by IT engineers for analysis, analyze it, and finally pass it on to decision-makers for decision-making.

To do this, the dataset itself must first be understood. For this reason, you first get an overview, as well as various impressions from different perspectives in order to understand the available data.

Our first view of the dataset indicates a dozen columns with both missing values as well as corrupted data. These pain points have to be eliminated since they could distort the picture we are about to draw with this data. We also need to make sure that the respective columns have the right format. Otherwise, they could block our way to the finish line.

We need to stem some terms and consolidate the dataset to have them in a way we can draw conclusions from the data:

In this task, we were asked to find relationships in the dataset to set the groundwork for determining the creditworthiness of the customer.

Therefore, several indicators are taken into account when looking for creditworthiness. A few questions have to be addressed to the customer data when requesting a loan:

1. How high is the income?
2. How old is the customer?
3. The purpose of the loan, as a loan for real estate is usually higher and lasts longer in repayment than a loan for a wedding.
4. Has the customer failed to repay on time in the past?
5. How strong is the household income?
6. ...

As there are relationships between this information, each of these pieces of information impacts creditworthiness. At a high level: The higher the household income and the lower the recurring monthly costs of the customer, the more likely their creditworthiness will improve.

However, the majority of customers appear creditworthy. Here's why: Many people have a solid income, are married with few or no children, and finally provide only a small risk of loan default. The most frequently given purpose was related to real estate.